In [1]:
import pandas as pd
import numpy as np
from calendar import monthrange

In [2]:
twitch_df = pd.read_csv('Twitch_game_data.csv', encoding='cp1252')

In [3]:
twitch_df.head()

,Rank,Game,Month,Year,Hours_watched,Hours_streamed,Peak_viewers,Peak_channels,Streamers,Avg_viewers,Avg_channels,Avg_viewer_ratio
0,1,League of Legends,1,2016,94377226,1362044,530270,2903,129172,127021,1833,69.29
1,2,Counter-Strike: Global Offensive,1,2016,47832863,830105,372654,2197,120849,64378,1117,57.62
2,3,Dota 2,1,2016,45185893,433397,315083,1100,44074,60815,583,104.26
3,4,Hearthstone,1,2016,39936159,235903,131357,517,36170,53749,317,169.29
4,5,Call of Duty: Black Ops III,1,2016,16153057,1151578,71639,3620,214054,21740,1549,14.03


In [4]:
twitch_df_global = pd.read_csv('Twitch_global_data.csv', encoding='cp1252')

In [5]:
twitch_df_global.head()

,year,Month,Hours_watched,Avg_viewers,Peak_viewers,Streams,Avg_channels,Games_streamed,Viewer_ratio
0,2016,1,480241904,646355,1275257,7701675,20076,12149,29.08
1,2016,2,441859897,635769,1308032,7038520,20427,12134,28.98
2,2016,3,490669308,660389,1591551,7390957,20271,12234,28.92
3,2016,4,377975447,525696,1775120,6869719,16791,12282,28.80
4,2016,5,449836631,605432,1438962,7535519,19394,12424,28.85


In [6]:
twitch_df.describe()

,Rank,Month,Year,Hours_watched,Hours_streamed,Peak_viewers,Peak_channels,Streamers,Avg_viewers,Avg_channels,Avg_viewer_ratio
count,17400.000000,17400.000000,17400.000000,1.740000e+04,1.740000e+04,1.740000e+04,17400.000000,1.740000e+04,17400.000000,17400.000000,17400.000000
mean,100.500000,6.344828,2019.137931,5.406489e+06,1.740947e+05,6.129193e+04,643.386207,2.082847e+04,7418.131437,238.397586,76.981366
std,57.735964,3.493370,2.096430,1.847645e+07,5.546933e+05,1.479578e+05,2677.160903,6.298185e+04,25362.412291,760.923850,339.203022
min,1.000000,1.000000,2016.000000,8.981100e+04,1.900000e+01,4.410000e+02,1.000000,0.000000e+00,120.000000,0.000000,1.670000
25%,50.750000,3.000000,2017.000000,4.329688e+05,1.414950e+04,9.270000e+03,58.000000,1.724750e+03,593.000000,19.000000,16.317500
50%,100.500000,6.000000,2019.000000,9.594830e+05,3.669700e+04,2.228400e+04,141.000000,4.795000e+03,1315.000000,50.000000,29.340000
75%,150.250000,9.000000,2021.000000,2.690568e+06,9.990650e+04,5.158575e+04,357.000000,1.271675e+04,3695.000000,136.000000,57.257500
max,200.000000,12.000000,2023.000000,3.445520e+08,1.024570e+07,3.366021e+06,129860.000000,1.013324e+06,479209.000000,13789.000000,13601.870000


In [7]:
'''RNC and DNC are single events, I need to look out for other single events and remove them from the dataset.'''

'RNC and DNC are single events, I need to look out for other single events and remove them from the dataset.'

In [8]:
twitch_df[twitch_df['Peak_channels'] == 1]

,Rank,Game,Month,Year,Hours_watched,Hours_streamed,Peak_viewers,Peak_channels,Streamers,Avg_viewers,Avg_channels,Avg_viewer_ratio
1329,130,RNC 2016,7,2016,264303,28,29021,1,2,355,0,9439.39


In [9]:
twitch_df[twitch_df['Game'] == "DNC 2016"]

,Rank,Game,Month,Year,Hours_watched,Hours_streamed,Peak_viewers,Peak_channels,Streamers,Avg_viewers,Avg_channels,Avg_viewer_ratio
1287,88,DNC 2016,7,2016,362702,355,47549,24,59,488,0,1021.7


In [10]:
twitch_df.drop([1329, 1287], inplace = True)
twitch_df.reset_index(inplace = True, drop = True)

In [11]:
#game_list is long, but you can review after downloading code and typing it. Omitted for ease of reading notebook. Summary below

In [12]:
game_list = list(set(list(twitch_df['Game'])))




In [13]:
'''There were a number of things to review/fix from the the game list on twitch
-- all E3 and twitchcon events are single events and would skew data
-- special characters not appearing correctly are to be fixed
-- review near duplicate games, Final Fantasy XIV and Legend of Zelda Games have some titles that are almost identical. 
    
    As an example there are three versions of Legend of Zelda: Ocarina of time. After some research these are still being streamed
    and tagged as different games so these will not be combined in any fashion. With a similar rationale I did not combine others
    
    Final Fantasy XIV: Online has several different iterations of names. Most are just a function of capitalization and need to
    be consistent. Final Fantasy XIV: Online was chosen. There was one instance of Final Fantasy XIV: Heavensward overlapping 
    with Final Fantasy XIV: Online. If it was just a matter of this one overlapping month I would have left it as is. But given
    that Heavensward is simply an expansion of Final Fantasy Online, Final Fantasy XIV: Heavensward will be changed to 
    Final Fantasy XIV: Online and the one month of overlapping data will be combined. This is because otherwise this would be 
    erroneously categorized as having dropped out of the top 200
    
'''

'There were a number of things to review/fix from the the game list on twitch\n-- all E3 and twitchcon events are single events and would skew data\n-- special characters not appearing correctly are to be fixed\n-- review near duplicate games, Final Fantasy XIV and Legend of Zelda Games have some titles that are almost identical. \n    \n    As an example there are three versions of Legend of Zelda: Ocarina of time. After some research these are still being streamed\n    and tagged as different games so these will not be combined in any fashion. With a similar rationale I did not combine others\n    \n    Final Fantasy XIV: Online has several different iterations of names. Most are just a function of capitalization and need to\n    be consistent. Final Fantasy XIV: Online was chosen. There was one instance of Final Fantasy XIV: Heavensward overlapping \n    with Final Fantasy XIV: Online. If it was just a matter of this one overlapping month I would have left it as is. But given\n    t

In [14]:
twitch_df[twitch_df['Game'].str.contains('TwitchCon') == True]

,Rank,Game,Month,Year,Hours_watched,Hours_streamed,Peak_viewers,Peak_channels,Streamers,Avg_viewers,Avg_channels,Avg_viewer_ratio
1761,164,TwitchCon 2016,9,2016,119450,166,44758,37,75,166,0,719.58
1848,51,TwitchCon 2016,10,2016,829794,725,125427,40,155,1116,0,1144.54
4259,62,TwitchCon 2017,10,2017,878473,1602,86285,89,411,1182,2,548.36


In [15]:
twitch_df.drop([1761,1848,4259], inplace = True)

In [16]:
ff_df = twitch_df[twitch_df['Game'].str.contains('XIV') == True]
ff_df.head()

,Rank,Game,Month,Year,Hours_watched,Hours_streamed,Peak_viewers,Peak_channels,Streamers,Avg_viewers,Avg_channels,Avg_viewer_ratio
111,112,Final Fantasy XIV: Heavensward,1,2016,305520,33688,13084,153,6456,411,45,9.07
284,85,Final Fantasy XIV: Heavensward,2,2016,358212,37285,3246,249,6468,515,53,9.61
464,65,Final Fantasy XIV: Heavensward,3,2016,598914,43341,17436,193,6615,806,58,13.82
688,89,Final Fantasy XIV: Heavensward,4,2016,319118,39940,5466,201,5694,443,55,7.99
885,86,Final Fantasy XIV: Heavensward,5,2016,305008,36029,10178,187,5575,410,48,8.47


In [17]:
ff_list = list(set(list(ff_df['Game'])))

In [18]:
ff_list

['Final Fantasy XIV Online',
 'Romance of the Three Kingdoms XIV',
 'The King of Fighters XIV',
 'FINAL FANTASY XIV ONLINE',
 'Final Fantasy XIV: Heavensward',
 'FINAL FANTASY XIV Online']

In [19]:
ff_list.remove('Romance of the Three Kingdoms XIV')
ff_list.remove('The King of Fighters XIV')

In [20]:
ff_list

['Final Fantasy XIV Online',
 'FINAL FANTASY XIV ONLINE',
 'Final Fantasy XIV: Heavensward',
 'FINAL FANTASY XIV Online']

In [21]:
twitch_df.reset_index(drop = True, inplace = True)

In [22]:
twitch_df = twitch_df.replace(ff_list, 'Final Fantasy XIV: Online')

In [23]:
e3_df = twitch_df[twitch_df['Game'].str.contains('E3') == True]
e3_df.index

Int64Index([1005, 3402, 3497, 5801, 5848, 5896], dtype='int64')

In [24]:
twitch_df.drop(e3_df.index, inplace = True)

In [25]:
twitch_df.drop(twitch_df[twitch_df['Game'] == 'Twitch Presents'].index, inplace = True)
twitch_df.reset_index(inplace = True, drop = True)

In [26]:
'''One thing that surprises me is the amount of classic games in this dataset. This will change the nature of the analysis in 
that games that are popular are not necessarily new so having a recent release date will likely negatively impact future 
popularity. 
'''

'One thing that surprises me is the amount of classic games in this dataset. This will change the nature of the analysis in \nthat games that are popular are not necessarily new so having a recent release date will likely negatively impact future \npopularity. \n'

In [27]:
twitch_df[twitch_df['Game'] =='Phoenix Wright: Ace Attorney <U+2212> Spirit of Justice']

,Rank,Game,Month,Year,Hours_watched,Hours_streamed,Peak_viewers,Peak_channels,Streamers,Avg_viewers,Avg_channels,Avg_viewer_ratio
1791,196,Phoenix Wright: Ace Attorney <U+2212> Spirit o...,9,2016,96620,842,2287,8,56,134,1,114.75


In [28]:
twitch_df = twitch_df.replace('Phoenix Wright: Ace Attorney <U+2212> Spirit of Justice', 'Phoenix Wright: Ace Attorney - Spirit of Justice')

In [29]:
twitch_df[twitch_df['Game'] =='Phoenix Wright: Ace Attorney - Spirit of Justice']

,Rank,Game,Month,Year,Hours_watched,Hours_streamed,Peak_viewers,Peak_channels,Streamers,Avg_viewers,Avg_channels,Avg_viewer_ratio
1791,196,Phoenix Wright: Ace Attorney - Spirit of Justice,9,2016,96620,842,2287,8,56,134,1,114.75


In [30]:
poke_df = twitch_df[twitch_df['Game'].str.contains('Pok') == True]
poke_df.head(10)

,Rank,Game,Month,Year,Hours_watched,Hours_streamed,Peak_viewers,Peak_channels,Streamers,Avg_viewers,Avg_channels,Avg_viewer_ratio
21,22,Poker,1,2016,3151063,36445,18034,126,2527,4241,49,86.46
50,51,Pok<U+00E9>mon Omega Ruby/Alpha Sapphire,1,2016,992216,20237,11278,61,1428,1335,27,49.03
78,79,Pok<U+00E9>mon Red/Blue,1,2016,456370,3276,185106,40,540,614,4,139.31
79,80,Pok<U+00E9>mon Yellow,1,2016,449878,855,210369,40,205,605,1,526.17
226,27,Poker,2,2016,2465052,36936,14039,115,2458,3546,53,66.74
252,53,Pok<U+00E9>mon Omega Ruby/Alpha Sapphire,2,2016,1014081,17894,16439,59,1510,1459,25,56.67
321,122,Pok<U+00E9>mon Red/Blue,2,2016,192855,4838,20162,53,865,277,6,39.86
377,178,Pok<U+00E9>mon FireRed/LeafGreen,2,2016,117710,3260,18360,21,803,169,4,36.11
422,23,Poker,3,2016,3534525,41966,25720,176,2592,4757,56,84.22
444,45,Pok<U+00E9>mon Omega Ruby/Alpha Sapphire,3,2016,1246547,24192,6980,64,1861,1677,32,51.53


In [31]:
poke_list = list(set(list(poke_df['Game'])))

In [32]:
poke_list.remove('Poker')
poke_list.remove('Prominence Poker')


In [33]:
poke_list[1]

'Pok<U+00E9>mon Ruby/Sapphire'

In [34]:
twitch_df.reset_index(drop = True, inplace = True)

In [35]:
def clean (string):
    lst = []
    for letter in string:
        lst.append(letter)
    if '<' in lst:
        index = lst.index('<')
        del lst[index:index + 8]
        lst.insert(index, 'e')
        lst = ''.join(lst)
        return(lst)
    if 'Ã' in lst:
        index = lst.index('Ã')
        del lst[index:index + 2]
        lst.insert(index, 'e')
        lst = ''.join(lst)
        return(lst)
    else:
        lst = ''.join(lst)
        return(lst)

In [36]:
x = len(twitch_df)
for i in range(x):
    twitch_df['Game'][i] = str(twitch_df['Game'][i])
    twitch_df['Game'][i] = clean(twitch_df['Game'][i])

C:\Users\danie\AppData\Local\Temp\ipykernel_28972\1143704737.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  twitch_df['Game'][i] = str(twitch_df['Game'][i])
C:\Users\danie\AppData\Local\Temp\ipykernel_28972\1143704737.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  twitch_df['Game'][i] = clean(twitch_df['Game'][i])


In [37]:
poke_list[0]

'Pok<U+00E9>mon Unite'

In [38]:
poke_df = twitch_df[twitch_df['Game'].str.contains('Pok') == True]
poke_df.head(10)

,Rank,Game,Month,Year,Hours_watched,Hours_streamed,Peak_viewers,Peak_channels,Streamers,Avg_viewers,Avg_channels,Avg_viewer_ratio
21,22,Poker,1,2016,3151063,36445,18034,126,2527,4241,49,86.46
50,51,Pokemon Omega Ruby/Alpha Sapphire,1,2016,992216,20237,11278,61,1428,1335,27,49.03
78,79,Pokemon Red/Blue,1,2016,456370,3276,185106,40,540,614,4,139.31
79,80,Pokemon Yellow,1,2016,449878,855,210369,40,205,605,1,526.17
226,27,Poker,2,2016,2465052,36936,14039,115,2458,3546,53,66.74
252,53,Pokemon Omega Ruby/Alpha Sapphire,2,2016,1014081,17894,16439,59,1510,1459,25,56.67
321,122,Pokemon Red/Blue,2,2016,192855,4838,20162,53,865,277,6,39.86
377,178,Pokemon FireRed/LeafGreen,2,2016,117710,3260,18360,21,803,169,4,36.11
422,23,Poker,3,2016,3534525,41966,25720,176,2592,4757,56,84.22
444,45,Pokemon Omega Ruby/Alpha Sapphire,3,2016,1246547,24192,6980,64,1861,1677,32,51.53


In [39]:
twitch_df[twitch_df['Game'].str.contains('Pokemon') == True]

,Rank,Game,Month,Year,Hours_watched,Hours_streamed,Peak_viewers,Peak_channels,Streamers,Avg_viewers,Avg_channels,Avg_viewer_ratio
50,51,Pokemon Omega Ruby/Alpha Sapphire,1,2016,992216,20237,11278,61,1428,1335,27,49.03
78,79,Pokemon Red/Blue,1,2016,456370,3276,185106,40,540,614,4,139.31
79,80,Pokemon Yellow,1,2016,449878,855,210369,40,205,605,1,526.17
252,53,Pokemon Omega Ruby/Alpha Sapphire,2,2016,1014081,17894,16439,59,1510,1459,25,56.67
321,122,Pokemon Red/Blue,2,2016,192855,4838,20162,53,865,277,6,39.86
...,...,...,...,...,...,...,...,...,...,...,...,...
17315,134,Pokemon UNITE,3,2023,1107267,67224,6369,221,8312,1490,90,16.47
17337,156,Pokemon Omega Ruby/Alpha Sapphire,3,2023,892623,2059,54088,15,506,1201,2,433.52
17339,158,Pokemon Trading Card Game,3,2023,888209,26741,4804,129,3172,1195,35,33.22
17371,190,Pokemon FireRed/LeafGreen,3,2023,649241,21671,6970,75,3237,873,29,29.96


In [40]:
twitch_df.head()

,Rank,Game,Month,Year,Hours_watched,Hours_streamed,Peak_viewers,Peak_channels,Streamers,Avg_viewers,Avg_channels,Avg_viewer_ratio
0,1,League of Legends,1,2016,94377226,1362044,530270,2903,129172,127021,1833,69.29
1,2,Counter-Strike: Global Offensive,1,2016,47832863,830105,372654,2197,120849,64378,1117,57.62
2,3,Dota 2,1,2016,45185893,433397,315083,1100,44074,60815,583,104.26
3,4,Hearthstone,1,2016,39936159,235903,131357,517,36170,53749,317,169.29
4,5,Call of Duty: Black Ops III,1,2016,16153057,1151578,71639,3620,214054,21740,1549,14.03


In [41]:
#As a result of fixing the pokemon games God of War Ragnarok was changed, but still incorrect. The below fixes this

In [42]:
gow_df = twitch_df[twitch_df['Game'].str.contains('Ragnarek') == True]

In [43]:
gow_df

,Rank,Game,Month,Year,Hours_watched,Hours_streamed,Peak_viewers,Peak_channels,Streamers,Avg_viewers,Avg_channels,Avg_viewer_ratio
16392,11,God of War Ragnarek,11,2022,40440752,1617975,480655,11583,136899,56245,2250,24.99
16637,56,God of War Ragnarek,12,2022,3798888,477077,34187,1673,65383,5112,642,7.96
16869,88,God of War Ragnarek,1,2023,1727833,257951,26831,866,40012,2325,347,6.70
17175,194,God of War Ragnarek,2,2023,552004,122533,7011,449,22703,822,182,4.50


In [44]:
twitch_df = twitch_df.replace('God of War Ragnarek', 'God of War Ragnarok')

In [45]:
#As a result of fixing the pokemon games Okami was changed, but incorrectly, the below fixes that

In [46]:
okami_df = twitch_df[twitch_df['Game'].str.contains('kami') == True]
okami_df

,Rank,Game,Month,Year,Hours_watched,Hours_streamed,Peak_viewers,Peak_channels,Streamers,Avg_viewers,Avg_channels,Avg_viewer_ratio
4773,185,ekami,12,2017,181539,19273,29429,148,3305,244,25,9.42
13887,106,Demon Slayer -Kimetsu no Yaiba- The Hinokami C...,10,2021,1641397,93397,57275,950,15889,2209,125,17.57


In [47]:
twitch_df = twitch_df.replace('ekami', 'Okami')

In [48]:
twitch_df.reset_index(drop = True, inplace = True)

In [49]:
twitch_df = twitch_df.sort_values(by = ['Game', 'Year', 'Month'])
twitch_df.head(10)
twitch_df.reset_index(drop = True, inplace = True)

In [50]:

x = len(twitch_df)
dup_l = []
for i in range(x-1):
    if twitch_df['Game'][i] == twitch_df['Game'][i+1] and twitch_df['Month'][i] == twitch_df['Month'][i+1] and twitch_df['Year'][i] == twitch_df['Year'][i+1]:
        dup_l.append(i)

In [51]:
dup_l

[518, 872, 3894, 4970, 11111, 11120, 11173, 11289, 11291, 13076]

In [52]:
examine = twitch_df.filter(items = dup_l, axis=0)
examine = examine.sort_values(by = ['Game', 'Year', 'Month'])

In [53]:
'''As we can see by the dataframe below there are a lot of duplicates, these will also have to be combined in some fashion.
One challenge in this is that Peak_viewers, Peak_channels, and Streamers are not computable. 
I will choose the max value of each. 

For consistency I should compute the average viewers, channels, and viewer_ratio based 
on the other columns in the dataframe for all columns.'''

'As we can see by the dataframe below there are a lot of duplicates, these will also have to be combined in some fashion.\nOne challenge in this is that Peak_viewers, Peak_channels, and Streamers are not computable. \nI will choose the max value of each. \n\nFor consistency I should compute the average viewers, channels, and viewer_ratio based \non the other columns in the dataframe for all columns.'

In [54]:
examine

,Rank,Game,Month,Year,Hours_watched,Hours_streamed,Peak_viewers,Peak_channels,Streamers,Avg_viewers,Avg_channels,Avg_viewer_ratio
518,68,Among Us,1,2023,2760266,76925,137287,342,37885,3715,103,35.88
872,15,Atlas,1,2019,12110031,187032,69614,637,10070,16298,251,64.75
3894,75,Dungeons & Dragons,9,2018,790966,21961,38597,181,2876,1100,30,36.02
4970,86,Final Fantasy XIV: Online,5,2017,533590,28073,35091,189,3965,718,37,19.01
11111,116,Resident Evil,10,2016,251183,3976,9871,21,7420,338,5,63.17
11120,13,Resident Evil 2,1,2019,16394917,449333,314244,6488,70644,22065,604,36.49
11173,11,Resident Evil 4,3,2023,27426481,1054133,327946,12902,137994,36913,1418,26.02
11289,72,Retro,9,2021,2976840,131077,20554,424,8577,4140,182,22.71
11291,83,Retro,10,2021,2484523,97914,10543,321,8118,3343,131,25.37
13076,27,Stray,7,2022,11428809,508590,287683,7662,136947,15381,684,22.47


In [55]:
new_values= []
def add_col(col):
    new_values.clear()
    for i in dup_l:
        new_values.append(twitch_df[col][i] + twitch_df[col][i+1])
        print(twitch_df[col][i], twitch_df[col][i+1])
    


In [56]:
add_col('Hours_watched')

2760266 876724
12110031 2106560
790966 550645
533590 450588
251183 143542
16394917 578746
27426481 2213184
2976840 1511016
2484523 1719684
11428809 909173


In [57]:
x = len(new_values)
def replace_cols(cols):
    for i in range(x):
        twitch_df.replace(twitch_df[cols][dup_l[i]], new_values[i], inplace = True)
        print(dup_l[i], new_values[i])


In [58]:
replace_cols('Hours_watched')

518 3636990
872 14216591
3894 1341611
4970 984178
11111 394725
11120 16973663
11173 29639665
11289 4487856
11291 4204207
13076 12337982


In [59]:
twitch_df['Hours_watched'][518]

3636990

In [60]:
add_col('Hours_streamed')

76925 14099
187032 23485
21961 10198
28073 34569
3976 4113
449333 16569
1054133 52760
131077 56192
97914 65164
508590 3856


In [61]:
replace_cols('Hours_streamed')

518 91024
872 210517
3894 32159
4970 62642
11111 8089
11120 465902
11173 1106893
11289 187269
11291 163078
13076 512446


In [62]:
def higher(col):
    new_values.clear()
    for i in dup_l:
        if twitch_df[col][i] > twitch_df[col][i+1]:
            new_values.append(twitch_df[col][i])
        else:
            new_values.append(twitch_df[col][i+1])

In [63]:
higher('Peak_viewers')

In [64]:
replace_cols('Peak_viewers')

518 137287
872 75774
3894 40261
4970 35091
11111 9871
11120 314244
11173 327946
11289 23532
11291 12134
13076 287683


In [65]:
higher('Peak_channels')

In [66]:
higher('Streamers')

In [67]:
'''
(Hours watched/hours in the month) is approximately the same as the current Avg_viewers; 
(Hours streamed/hours in the month) is approximately the same as Avg_channels;

Avg_viewer_ratio is (Avg_viewers/Avg_channels); however when computing with the values I found that I was sometimes dividing
by 0. To use these values I simply reversed the division to Avg_channels/Avg_viewers.

I will be using these values instead given there are some duplicates not of my own making that needed combined.
Also, the data card was not forthcoming as to the provenance of these columns. Creating my own similar metrics
makes sense. 

Rank will not be helpful at this point so I will remove'''

'\n(Hours watched/hours in the month) is approximately the same as the current Avg_viewers; \n(Hours streamed/hours in the month) is approximately the same as Avg_channels;\n\nAvg_viewer_ratio is (Avg_viewers/Avg_channels); however when computing with the values I found that I was sometimes dividing\nby 0. To use these values I simply reversed the division to Avg_channels/Avg_viewers.\n\nI will be using these values instead given there are some duplicates not of my own making that needed combined.\nAlso, the data card was not forthcoming as to the provenance of these columns. Creating my own similar metrics\nmakes sense. \n\nRank will not be helpful at this point so I will remove'

In [68]:
no_leap = []
leap = []
for i in range(1,13):
    leap.append(monthrange(2004, i)[1] * 24)
    no_leap.append(monthrange(2003, i)[1] * 24)

In [69]:
no_leap

[744, 672, 744, 720, 744, 720, 744, 744, 720, 744, 720, 744]

In [70]:
leap

[744, 696, 744, 720, 744, 720, 744, 744, 720, 744, 720, 744]

In [71]:
x = len(twitch_df)
month_hours = []
for i in range(x):
    if twitch_df['Year'][i] %4:
        hours = leap[(twitch_df['Month'][i]-1)]
        month_hours.append(hours)
    else:
        hours = no_leap[(twitch_df['Month'][i]-1)]
        month_hours.append(hours)

In [72]:
twitch_df['Month_hours'] = month_hours
twitch_df.head()

,Rank,Game,Month,Year,Hours_watched,Hours_streamed,Peak_viewers,Peak_channels,Streamers,Avg_viewers,Avg_channels,Avg_viewer_ratio,Month_hours
0,199,.hack//G.U. Last Recode,11,2017,145350,35258,1222,258,3174,202,49,4.12,720
1,159,20 Minutes Till Dawn,6,2022,911356,12253,29743,80,3616,1267,17,74.38,720
2,109,60 Parsecs!,9,2018,529688,1867,31960,27,606,736,2,283.71,720
3,126,60 Seconds!,7,2016,268754,597,32505,10,275,361,0,450.17,744
4,54,60 Seconds!,8,2016,772786,2065,56904,30,628,1040,2,374.23,744


In [73]:
twitch_df['Avg_viewers'] = round(twitch_df['Hours_watched']/twitch_df['Month_hours'])
twitch_df.head()

,Rank,Game,Month,Year,Hours_watched,Hours_streamed,Peak_viewers,Peak_channels,Streamers,Avg_viewers,Avg_channels,Avg_viewer_ratio,Month_hours
0,199,.hack//G.U. Last Recode,11,2017,145350,35258,1222,258,3174,202.0,49,4.12,720
1,159,20 Minutes Till Dawn,6,2022,911356,12253,29743,80,3616,1266.0,17,74.38,720
2,109,60 Parsecs!,9,2018,529688,1867,31960,27,606,736.0,2,283.71,720
3,126,60 Seconds!,7,2016,268754,597,32505,10,275,361.0,0,450.17,744
4,54,60 Seconds!,8,2016,772786,2065,56904,30,628,1039.0,2,374.23,744


In [74]:
twitch_df['Avg_channels'] = round(twitch_df['Hours_streamed']/twitch_df['Month_hours'])
twitch_df.head()

,Rank,Game,Month,Year,Hours_watched,Hours_streamed,Peak_viewers,Peak_channels,Streamers,Avg_viewers,Avg_channels,Avg_viewer_ratio,Month_hours
0,199,.hack//G.U. Last Recode,11,2017,145350,35258,1222,258,3174,202.0,49.0,4.12,720
1,159,20 Minutes Till Dawn,6,2022,911356,12253,29743,80,3616,1266.0,17.0,74.38,720
2,109,60 Parsecs!,9,2018,529688,1867,31960,27,606,736.0,3.0,283.71,720
3,126,60 Seconds!,7,2016,268754,597,32505,10,275,361.0,1.0,450.17,744
4,54,60 Seconds!,8,2016,772786,2065,56904,30,628,1039.0,3.0,374.23,744


In [75]:
twitch_df['Avg_channel_ratio'] = round(twitch_df['Avg_channels']/twitch_df['Avg_viewers'],2)
twitch_df.head()

,Rank,Game,Month,Year,Hours_watched,Hours_streamed,Peak_viewers,Peak_channels,Streamers,Avg_viewers,Avg_channels,Avg_viewer_ratio,Month_hours,Avg_channel_ratio
0,199,.hack//G.U. Last Recode,11,2017,145350,35258,1222,258,3174,202.0,49.0,4.12,720,0.24
1,159,20 Minutes Till Dawn,6,2022,911356,12253,29743,80,3616,1266.0,17.0,74.38,720,0.01
2,109,60 Parsecs!,9,2018,529688,1867,31960,27,606,736.0,3.0,283.71,720,0.00
3,126,60 Seconds!,7,2016,268754,597,32505,10,275,361.0,1.0,450.17,744,0.00
4,54,60 Seconds!,8,2016,772786,2065,56904,30,628,1039.0,3.0,374.23,744,0.00


In [76]:
twitch_df.drop(['Rank', 'Month_hours', 'Avg_viewer_ratio'], axis = 1, inplace = True)


In [77]:
twitch_df.drop(twitch_df[twitch_df['Game'] == 'Twitch Presents'].index, inplace = True)
twitch_df.reset_index(inplace = True, drop = True)

In [78]:
twitch_df.sort_values(by = ['Month', 'Year', 'Game'], )
twitch_df.head(20)

,Game,Month,Year,Hours_watched,Hours_streamed,Peak_viewers,Peak_channels,Streamers,Avg_viewers,Avg_channels,Avg_channel_ratio
0,.hack//G.U. Last Recode,11,2017,145350,35258,1222,258,3174,202.0,49.0,0.24
1,20 Minutes Till Dawn,6,2022,911356,12253,29743,80,3616,1266.0,17.0,0.01
2,60 Parsecs!,9,2018,529688,1867,31960,27,606,736.0,3.0,0.00
3,60 Seconds!,7,2016,268754,597,32505,10,275,361.0,1.0,0.00
4,60 Seconds!,8,2016,772786,2065,56904,30,628,1039.0,3.0,0.00
5,60 Seconds!,11,2016,109473,503,5297,5,193,152.0,1.0,0.01
6,60 Seconds!,3,2017,263482,679,31311,10,240,354.0,1.0,0.00
7,60 Seconds!,11,2017,179250,584,10458,8,228,249.0,1.0,0.00
8,60 Seconds!,12,2017,180788,669,23476,9,260,243.0,1.0,0.00
9,60 Seconds!,8,2018,222209,871,18245,8,414,299.0,1.0,0.00


In [82]:
df = list(twitch_df.Year.astype(str) + '/' + twitch_df.Month.astype(str) + '/01')
twitch_df['Date'] = df
twitch_df['Date']= pd.to_datetime(twitch_df['Date'])

In [83]:
twitch_df_X = twitch_df.drop_duplicates(subset=['Game'])
twitch_df_X.head(10)

,Game,Month,Year,Hours_watched,Hours_streamed,Peak_viewers,Peak_channels,Streamers,Avg_viewers,Avg_channels,Avg_channel_ratio,Date
0,.hack//G.U. Last Recode,11,2017,145350,35258,1222,258,3174,202.0,49.0,0.24,2017-11-01
1,20 Minutes Till Dawn,6,2022,911356,12253,29743,80,3616,1266.0,17.0,0.01,2022-06-01
2,60 Parsecs!,9,2018,529688,1867,31960,27,606,736.0,3.0,0.00,2018-09-01
3,60 Seconds!,7,2016,268754,597,32505,10,275,361.0,1.0,0.00,2016-07-01
12,7 Days to Die,1,2016,269681,12131,4405,44,1240,362.0,16.0,0.04,2016-01-01
99,A Dance of Fire and Ice,2,2019,509018,5428,19067,48,2071,731.0,8.0,0.01,2019-02-01
101,A Hat in Time,10,2017,488213,20147,28625,182,3017,656.0,27.0,0.04,2017-10-01
104,A Plague Tale: Innocence,5,2019,2632781,54353,63953,608,7837,3539.0,73.0,0.02,2019-05-01
106,A Plague Tale: Requiem,10,2022,4244194,132754,117084,1820,20316,5705.0,178.0,0.03,2022-10-01
108,A Total War Saga: Troy,8,2020,1019391,23357,54218,415,3543,1370.0,31.0,0.02,2020-08-01


In [84]:
twitch_df_X.describe()

,Month,Year,Hours_watched,Hours_streamed,Peak_viewers,Peak_channels,Streamers,Avg_viewers,Avg_channels,Avg_channel_ratio
count,2025.000000,2025.000000,2.025000e+03,2.025000e+03,2.025000e+03,2025.000000,2025.000000,2025.000000,2025.000000,2025.000000
mean,5.884444,2018.348148,1.945442e+06,4.446292e+04,6.342272e+04,407.241975,6539.648889,2668.952099,61.166420,0.021723
std,3.601367,2.185737,9.351513e+06,2.099008e+05,9.913581e+04,1307.981561,23237.997784,12988.438559,295.530889,0.027284
min,1.000000,2016.000000,9.246000e+04,1.900000e+01,7.470000e+02,2.000000,0.000000,126.000000,0.000000,0.000000
25%,2.000000,2016.000000,2.538660e+05,2.336000e+03,1.763700e+04,23.000000,405.000000,343.000000,3.000000,0.010000
50%,6.000000,2018.000000,5.805360e+05,8.944000e+03,3.535100e+04,77.000000,1573.000000,796.000000,12.000000,0.010000
75%,9.000000,2020.000000,1.357201e+06,2.737800e+04,7.232700e+04,265.000000,4863.000000,1864.000000,37.000000,0.030000
max,12.000000,2023.000000,3.445520e+08,5.580337e+06,1.832845e+06,29306.000000,663781.000000,478544.000000,8018.000000,0.280000


In [85]:
zero_df = twitch_df[twitch_df['Avg_channels'] == 0]
zero_df.head(11)

,Game,Month,Year,Hours_watched,Hours_streamed,Peak_viewers,Peak_channels,Streamers,Avg_viewers,Avg_channels,Avg_channel_ratio,Date
264,Age of Empires,1,2016,248884,232,107455,18,40,335.0,0.0,0.0,2016-01-01
613,Ape Escape,7,2016,166213,149,138243,8,46,223.0,0.0,0.0,2016-07-01
667,Arcanum: Of Steamworks and Magick Obscura,11,2019,310380,238,12070,3,36,431.0,0.0,0.0,2019-11-01
911,Azure Striker GUNVOLT,1,2016,197178,217,135933,14,71,265.0,0.0,0.0,2016-01-01
912,Azure Striker GUNVOLT 2,7,2017,291374,86,142476,8,20,392.0,0.0,0.0,2017-07-01
933,Barbie and The Magic of Pegasus,5,2016,97298,27,37158,8,22,131.0,0.0,0.0,2016-05-01
934,Barbie and The Magic of Pegasus,7,2016,163007,76,37926,8,42,219.0,0.0,0.0,2016-07-01
935,Barbie and The Magic of Pegasus,8,2016,181848,147,34944,11,64,244.0,0.0,0.0,2016-08-01
936,Barbie and The Magic of Pegasus,4,2017,140000,147,45811,10,60,194.0,0.0,0.0,2017-04-01
937,Barbie and The Magic of Pegasus,5,2017,180582,124,39137,7,52,243.0,0.0,0.0,2017-05-01


In [86]:
one_df = twitch_df[twitch_df['Avg_channels'] == 1]
one_df.head(11)

,Game,Month,Year,Hours_watched,Hours_streamed,Peak_viewers,Peak_channels,Streamers,Avg_viewers,Avg_channels,Avg_channel_ratio,Date
3,60 Seconds!,7,2016,268754,597,32505,10,275,361.0,1.0,0.00,2016-07-01
5,60 Seconds!,11,2016,109473,503,5297,5,193,152.0,1.0,0.01,2016-11-01
6,60 Seconds!,3,2017,263482,679,31311,10,240,354.0,1.0,0.00,2017-03-01
7,60 Seconds!,11,2017,179250,584,10458,8,228,249.0,1.0,0.00,2017-11-01
8,60 Seconds!,12,2017,180788,669,23476,9,260,243.0,1.0,0.00,2017-12-01
9,60 Seconds!,8,2018,222209,871,18245,8,414,299.0,1.0,0.00,2018-08-01
123,ARAYA,11,2016,142823,437,28289,12,184,198.0,1.0,0.01,2016-11-01
477,American Truck Simulator,1,2016,314055,724,43089,48,186,422.0,1.0,0.00,2016-01-01
614,Ape Escape,7,2017,299969,556,163153,11,72,403.0,1.0,0.00,2017-07-01
931,Banjo-Tooie,6,2018,272135,906,104527,11,131,378.0,1.0,0.00,2018-06-01


In [87]:
two_df = twitch_df[twitch_df['Avg_channels'] == 2]
two_df.head(11)

,Game,Month,Year,Hours_watched,Hours_streamed,Peak_viewers,Peak_channels,Streamers,Avg_viewers,Avg_channels,Avg_channel_ratio,Date
10,60 Seconds!,7,2019,435311,1188,41049,13,630,585.0,2.0,0.00,2019-07-01
252,Abandon Ship,2,2018,186349,1426,11183,36,392,268.0,2.0,0.01,2018-02-01
374,Age of Empires: Definitive Edition,8,2019,268536,1770,30412,21,362,361.0,2.0,0.01,2019-08-01
378,Agrou,8,2020,534353,1524,42718,19,318,718.0,2.0,0.00,2020-08-01
564,Animal Crossing: New Leaf,11,2016,117314,1399,13805,16,160,163.0,2.0,0.01,2016-11-01
583,Anime,8,2017,462889,1244,24296,74,440,622.0,2.0,0.00,2017-08-01
885,Atlas Reactor,2,2016,127027,1461,15418,24,304,189.0,2.0,0.01,2016-02-01
1053,Battlefield: Bad Company 2,6,2016,125333,1081,28282,12,388,174.0,2.0,0.01,2016-06-01
1103,Beholder,4,2018,209247,1492,7740,14,372,291.0,2.0,0.01,2018-04-01
1213,Blackjack,2,2016,168598,1062,28638,7,178,251.0,2.0,0.01,2016-02-01


In [91]:
plus_one = 1
X = twitch_df_X.Date + pd.DateOffset(months=plus_one)

In [92]:
twitch_df_X['one_month_future'] =  twitch_df_X.Date + pd.DateOffset(months=plus_one)

C:\Users\danie\AppData\Local\Temp\ipykernel_28972\2229078347.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  twitch_df_X['one_month_future'] =  twitch_df_X.Date + pd.DateOffset(months=plus_one)


In [93]:
twitch_df_X.head()

,Game,Month,Year,Hours_watched,Hours_streamed,Peak_viewers,Peak_channels,Streamers,Avg_viewers,Avg_channels,Avg_channel_ratio,Date,one_month_future
0,.hack//G.U. Last Recode,11,2017,145350,35258,1222,258,3174,202.0,49.0,0.24,2017-11-01,2017-12-01
1,20 Minutes Till Dawn,6,2022,911356,12253,29743,80,3616,1266.0,17.0,0.01,2022-06-01,2022-07-01
2,60 Parsecs!,9,2018,529688,1867,31960,27,606,736.0,3.0,0.00,2018-09-01,2018-10-01
3,60 Seconds!,7,2016,268754,597,32505,10,275,361.0,1.0,0.00,2016-07-01,2016-08-01
12,7 Days to Die,1,2016,269681,12131,4405,44,1240,362.0,16.0,0.04,2016-01-01,2016-02-01


In [95]:
plus_three = 3
X3 = twitch_df_X.Date + pd.DateOffset(months=plus_three)

In [96]:
twitch_df_X['three_month_future'] =  twitch_df_X.Date + pd.DateOffset(months=plus_three)

C:\Users\danie\AppData\Local\Temp\ipykernel_28972\4009328254.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  twitch_df_X['three_month_future'] =  twitch_df_X.Date + pd.DateOffset(months=plus_three)


In [98]:
twitch_df_X.head()

,Game,Month,Year,Hours_watched,Hours_streamed,Peak_viewers,Peak_channels,Streamers,Avg_viewers,Avg_channels,Avg_channel_ratio,Date,one_month_future,three_month_future
0,.hack//G.U. Last Recode,11,2017,145350,35258,1222,258,3174,202.0,49.0,0.24,2017-11-01,2017-12-01,2018-02-01
1,20 Minutes Till Dawn,6,2022,911356,12253,29743,80,3616,1266.0,17.0,0.01,2022-06-01,2022-07-01,2022-09-01
2,60 Parsecs!,9,2018,529688,1867,31960,27,606,736.0,3.0,0.00,2018-09-01,2018-10-01,2018-12-01
3,60 Seconds!,7,2016,268754,597,32505,10,275,361.0,1.0,0.00,2016-07-01,2016-08-01,2016-10-01
12,7 Days to Die,1,2016,269681,12131,4405,44,1240,362.0,16.0,0.04,2016-01-01,2016-02-01,2016-04-01


In [100]:
plus_six = 6
X6 = twitch_df_X.Date + pd.DateOffset(months=plus_six)

In [101]:
twitch_df_X['six_month_future'] =  twitch_df_X.Date + pd.DateOffset(months=plus_six)

C:\Users\danie\AppData\Local\Temp\ipykernel_28972\3018229530.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  twitch_df_X['six_month_future'] =  twitch_df_X.Date + pd.DateOffset(months=plus_six)


In [103]:
twitch_df_X.head()

,Game,Month,Year,Hours_watched,Hours_streamed,Peak_viewers,Peak_channels,Streamers,Avg_viewers,Avg_channels,Avg_channel_ratio,Date,one_month_future,three_month_future,six_month_future
0,.hack//G.U. Last Recode,11,2017,145350,35258,1222,258,3174,202.0,49.0,0.24,2017-11-01,2017-12-01,2018-02-01,2018-05-01
1,20 Minutes Till Dawn,6,2022,911356,12253,29743,80,3616,1266.0,17.0,0.01,2022-06-01,2022-07-01,2022-09-01,2022-12-01
2,60 Parsecs!,9,2018,529688,1867,31960,27,606,736.0,3.0,0.00,2018-09-01,2018-10-01,2018-12-01,2019-03-01
3,60 Seconds!,7,2016,268754,597,32505,10,275,361.0,1.0,0.00,2016-07-01,2016-08-01,2016-10-01,2017-01-01
12,7 Days to Die,1,2016,269681,12131,4405,44,1240,362.0,16.0,0.04,2016-01-01,2016-02-01,2016-04-01,2016-07-01


In [104]:
twitch_df_X.reset_index(drop = True, inplace = True)

In [117]:
#twitch_df_X one_mnth_hrs = twitch_df['hours_watched'] where twitch_df['Date'] == twitch_df one_month_future & twitch_df['Game']
'''
one_month = []
x = len(twitch_df)
y = len(twitch_df_X)
for i in range(x):
    for j in range(y):
        if twitch_df['Date'][i] == twitch_df_X['one_month_future'][j] and twitch_df['Game'][i] == twitch_df_X['Game'][j]:
            one_month.append(twitch_df['Hours_watched'][i])
'''